In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch

In [2]:
H = 400 # Height of an image
W = 400 # Width of an image
f = 1200 # Focal distance

In [3]:
rays_o = np.zeros((H*W, 3)) # Focal origin
rays_d = np.zeros((H*W, 3)) # Direction of each ray

u = np.arange(W)
v = np.arange(H)

u, v = np.meshgrid(u, v) # initialize image grid

dirs = np.stack((u - W / 2,
                 -(v - H / 2),
                 -np.ones_like(u) * f), axis=-1) # stack the 3 vectors together
rays_d = dirs / np.linalg.norm(dirs, axis=-1, keepdims=True) # normalize the direction vector
rays_d = rays_d.reshape(-1,3)

In [4]:
def plot_rays(o, d, t):
    fig = plt.figure(figsize=(12, 12))
    ax = plt.axes(projection='3d')

    pt1 = o
    pt2 = o + t * d # where the ray is after time t

    for p1, p2 in zip(pt1[::10], pt2[::10]): # merge 2 stuff together (pytorch)
        plt.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]])

    plt.show()

In [5]:
#plot_rays(rays_o, rays_d, 1)

In [6]:
class Sphere:
    def __init__(self, position, radius, color):
        self.position = position
        self.radius = radius
        self.color = color

    def intersect(self, x):
        # To make sure point x is inside the sphere so that it contains volume
        """
        :param x: [batch_size, 3]
        """
        cond = (x[:, 0] - self.position[0])**2 + (x[:, 1] - self.position[1])**2 + (x[:, 2] - self.position[2])**2 <= self.radius**2

        num_rays = x.shape[0] # length of first dimension
        colors = torch.zeros((num_rays, 3))
        density = torch.zeros((num_rays, 1))
        colors[cond] = self.color
        density[cond] = 10
        return colors, density

In [7]:
s = Sphere(np.array([0., 0., -1.]), .1, np.array([1., 0., 0.]))

In [8]:
def compute_accumulated_transmittance(betas):
    # Returns the cumulative product of elements of input in the dimension dim.
    accumulated_transmittance = torch.cumprod(betas, 1)
    return accumulated_transmittance


In [16]:
# tn, tf represent near and far bound
def rendering(model, rays_o, rays_d, tn, tf, num_bins=100, device='cpu'):
    t = torch.linspace(tn, tf, num_bins).to(device) # size: [num_bins]

    # 1: start from 1,
    # -1: start from 0, not include -1
    # delta_i is the distance between adjacent samples.
    delta = torch.cat((t[1:] - t[:-1], torch.tensor([1e10])))


    """
    shape:
    t: [num_bins]
    ray_o: [num_rays, 3]
    We want x: [num_rays, num_bins, 3]
    Because for each ray, we need num_bins points,
    the extra 3 dimensions contain position info.
    So in order to get that dimension, virtual dimensions are
    added like:
    t: [1, num_bins, 1]
    ray_o: [num_rays, 1, 3]
    !!!!x currently is many many empty rays with many bins!!!!
    we intersect the model with x, then we can get
    many rendered rays.
    """
    x = rays_o.unsqueeze(1) + t.unsqueeze(0).unsqueeze(-1) * rays_d.unsqueeze(1)
    # Now x: [batch_size, 3]


    colors, density = model.intersect(x.reshape(-1, 3))

    colors = colors.reshape((x.shape[0], num_bins, 3)) # [num_rays, num_bins, 3]
    density = density.reshape((x.shape[0], num_bins)) # [num_rays, num_bins]

    alpha = 1 - torch.exp(-density * delta.unsqueeze(0)) # [num_rays, num_bins]
    T = compute_accumulated_transmittance(1 - alpha)
    c = (T.unsqueeze(-1) * alpha.unsqueeze(-1) * colors).sum(1) # [num_rays, 3] because sum(1)

In [17]:
sphere_model = Sphere(torch.zeros(3), 0.1, torch.tensor([1., 0., 0.]))
rendering(sphere_model, torch.from_numpy(rays_o), torch.from_numpy(rays_d), 0.8, 1.2)

torch.Size([160000, 100])
torch.Size([160000, 100])
torch.Size([160000, 100, 3])
